<a href="https://colab.research.google.com/github/heejvely/third-project/blob/main/Etri_et5_train_120000_extra_training_ver4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

참고 url : https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_summarization_wandb.ipynb

In [ ]:
# mount
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# module install 
!pip install transformers
!pip install sentencepiece==0.1.91

     |████████████████████████████████| 3.1 MB 13.2 MB/s 
     |████████████████████████████████| 596 kB 83.8 MB/s 
     |████████████████████████████████| 895 kB 78.1 MB/s 
     |████████████████████████████████| 3.3 MB 65.2 MB/s 
     |████████████████████████████████| 59 kB 9.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.1 MB 15.9 MB/s 


런타임 다시 시작 눌러야할 수 있음  
만일 tokenizer Nonetype 에러시 런타임 다시 시작

In [ ]:
# model.generate(pieces)
from transformers import T5Config, T5Tokenizer, T5ForConditionalGeneration
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader

# model folder path
model_folder = '/content/drive/MyDrive/colab/3차/pretrained_train_120000_extra_training_ver3/'

model = T5ForConditionalGeneration.from_pretrained(model_folder)
tokenizer = T5Tokenizer.from_pretrained(model_folder)

In [ ]:
# gpu로 지정
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
class CustomDataset:

# """
# author : Author of the article
# date : Date the article was published
# headline: Headline for the published article
# read_more : URL for the article to follow online
# text: This is the summary of the article
# ctext: This is the complete article
# """

    def __init__(self, dataframe, tokenizer, source_len, summ_len):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_len = source_len
        self.summ_len = summ_len
        self.text = self.data.text
        self.ctext = self.data.ctext

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        ctext = str(self.ctext[index])
        ctext = ' '.join(ctext.split())

        text = str(self.text[index])
        text = ' '.join(text.split())

        source = self.tokenizer.batch_encode_plus([ctext], max_length= self.source_len, pad_to_max_length=True,return_tensors='pt')
        target = self.tokenizer.batch_encode_plus([text], max_length= self.summ_len, pad_to_max_length=True,return_tensors='pt')

        source_ids = source['input_ids'].squeeze()
        source_mask = source['attention_mask'].squeeze()
        target_ids = target['input_ids'].squeeze()
        target_mask = target['attention_mask'].squeeze()

        return {
            'source_ids': source_ids.to(dtype=torch.long), 
            'source_mask': source_mask.to(dtype=torch.long), 
            'target_ids': target_ids.to(dtype=torch.long),
            'target_ids_y': target_ids.to(dtype=torch.long)
        }

In [ ]:
def train(epoch, tokenizer, model, device, loader, optimizer):
    model.train()
    for _,data in tqdm(enumerate(loader, 0)):
        y = data['target_ids'].to(device, dtype = torch.long)
        y_ids = y[:, :-1].contiguous()
        lm_labels = y[:, 1:].clone().detach()
        lm_labels[y[:, 1:] == tokenizer.pad_token_id] = -100
        ids = data['source_ids'].to(device, dtype = torch.long)
        mask = data['source_mask'].to(device, dtype = torch.long)

        outputs = model(input_ids = ids, attention_mask = mask, decoder_input_ids=y_ids, labels=lm_labels)
        loss = outputs[0]
        
        if _%10 == 0:
            pass
            
        if _%500==0:
            print(f'Epoch: {epoch}, Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # xm.optimizer_step(optimizer)
        # xm.mark_step()

In [ ]:
def validate(epoch, tokenizer, model, device, loader):
    model.eval()
    predictions = []
    actuals = []
    with torch.no_grad():
        for _, data in enumerate(loader, 0):
            y = data['target_ids'].to(device, dtype = torch.long)
            ids = data['source_ids'].to(device, dtype = torch.long)
            mask = data['source_mask'].to(device, dtype = torch.long)

            generated_ids = model.generate(
                input_ids = ids,
                attention_mask = mask, 
                max_length=150, 
                num_beams=2,
                repetition_penalty=2.5, 
                length_penalty=1.0, 
                early_stopping=True
                )
            preds = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=True) for g in generated_ids]
            target = [tokenizer.decode(t, skip_special_tokens=True, clean_up_tokenization_spaces=True)for t in y]
            if _%100==0:
                print(f'Completed {_}')

            predictions.extend(preds)
            actuals.extend(target)
    return predictions, actuals

In [ ]:
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(45100, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(45100, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedGeluDense(
              (wi_0): Linear(in_features=768, out_features=3072, bias=False)
              (wi_1): Linear(in_features=768, out_features=3072, bias=False)
              (wo)

hyper-parameters

In [ ]:
config = T5Config()
config.MAX_LEN = 1024
config.SUMMARY_LEN = 150 
config.TRAIN_BATCH_SIZE = 2    # input batch size for training (default: 64)
config.VALID_BATCH_SIZE = 2    # input batch size for testing (default: 1000)
config.TRAIN_EPOCHS = 15       # number of epochs to train (default: 10)
config.VAL_EPOCHS = 1 
config.LEARNING_RATE = 1e-4    # learning rate (default: 0.01)
config.SEED = 42               # random seed (default: 42)

In [ ]:
train_params = {
        'batch_size': config.TRAIN_BATCH_SIZE,
        'shuffle': True,
        'num_workers': 0
        }

val_params = {
        'batch_size': config.VALID_BATCH_SIZE,
        'shuffle': False,
        'num_workers': 0
        }

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)

In [ ]:
!nvidia-smi

Sun Nov 28 01:44:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    38W / 300W |   3015MiB / 16160MiB |     29%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import pandas as pd
train_dataset = pd.read_csv('/content/drive/MyDrive/colab/3차/dataset/120000_ver4.csv')[['document','label']]
validation_dataset = pd.read_csv('/content/drive/MyDrive/colab/3차/dataset/valid_500.csv')[['document','label']]

partial dataset


In [ ]:
import numpy as np

train_dataset = train_dataset
validation_dataset = validation_dataset

train

In [ ]:
train_dataset.columns = ['ctext','text']
train_dataset.ctext = 'summarize: ' + train_dataset.ctext

training_set = CustomDataset(train_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

training_loader = DataLoader(training_set, **train_params)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=config.LEARNING_RATE)
for epoch in range(config.TRAIN_EPOCHS):
    print (1)
    train(epoch, tokenizer, model, device, training_loader, optimizer)

1


0it [00:00, ?it/s]Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Epoch: 0, Loss:  2.2247934341430664


501it [02:41,  3.12it/s]

Epoch: 0, Loss:  1.7633365392684937


...

14001it [1:15:28,  3.10it/s]

Epoch: 14, Loss:  0.365206241607666


14501it [1:18:10,  3.09it/s]

Epoch: 14, Loss:  0.17553259432315826


15000it [1:20:51,  3.09it/s]


In [ ]:
# model save
tokenizer.save_pretrained('/content/drive/MyDrive/colab/3차/pretrained_train_120000_extra_training_ver4/')
model.save_pretrained('/content/drive/MyDrive/colab/3차/pretrained_train_120000_extra_training_ver4/')

valid(test)

In [ ]:
validation_dataset.columns = ['ctext','text']
validation_dataset.ctext = 'summarize: ' + validation_dataset.ctext

val_set = CustomDataset(validation_dataset, tokenizer, config.MAX_LEN, config.SUMMARY_LEN)

val_loader = DataLoader(val_set, **val_params)

for epoch in range(config.VAL_EPOCHS):
    predictions, actuals = validate(epoch, tokenizer, model, device, val_loader)
    final_df = pd.DataFrame({'Generated Text':predictions,'Actual Text':actuals})

final_df

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2218: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Completed 0
Completed 100
Completed 200


,Generated Text,Actual Text
0,스코 소속사 커넥트엔터테인먼트 측은 19일 공식 홈페이지를 통해 9월 초 법률 자문...,가수 강다니엘이 모욕죄 혐의로 악플러에 대한 고소장을 서울강남경찰서에 제출하며 합의...
1,으로 '2019년 안산시 청소년의회'가 의장단 구성을 마치고 본격적인 활동을 시작하...,안산시의회는 지난 7월 22일과 29일에 청소년의회 제1·2차 본회의를 열어 실제 ...
2,오는 2019 광주세계수영선수권대회에서 수구와 다이빙 등 12명의 여자선수들의 신체...,2019 광주세계수영선수권대회에서 18일 일본인A씨가 12명의 여자선수들의 신체 일...
3,책정된 익산시 선화로53길 8 일원에 위치한 ‘익산 오투그란데 글로벌카운티’ 모델하...,제일건설은 실수요자들에게 선호도 높은 전용 면적과 효율적인 공간 활용을 강점으로 1...
4,"시 00분 등락률 1위를 기록한 에이치엘비 종목은 전일대비 19,200원 상승한 1...",월요일 11시 00분 현재 조선업종 내 상승종목 등락률 1위를 차지한 에이치엘비종목...
...,...,...
495,산지인 거제도는 해금강부터 멋스러운 조형물과 이국적인 자연 풍경을 가득담은 거제 외...,식도락 여행을 즐기기에 안성맞춤인 천고마비의 계절 가을에는 천혜의 자연 경관을 자랑...
496,이는 3조원대에 이르면서 간편가정식 상품 시장이 3조원대에 이르면서 ‘간편식 밀리언...,1인 가구 증가 등에 따른 혼밥 문화가 확산되면서 간편가정식 상품 판매량이 100만...
497,"문화예술 공연의 관람정보를 영화처럼 한눈에 볼 수 있는 공연전산망이 가동된 가운데,...",문화관광체육부에 따르면 개편된 공연전산망덕분에 공연정보와 관련 통계정보를 확인하기 ...
498,리그서 '페어플레이 점수'를 앞세워 극락함을 이어가고 있는 베트남과 중국은 2019...,베트남은 2019 아시아 축구연맹 아시안컵에서 요르단과 120분 연장 혈투 끝에 8...


In [ ]:
final_df.to_csv('/content/drive/MyDrive/colab/3차/dataset/summarization_et5_train_120000_extra_training_ver4.csv')